In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from zipfile import ZipFile
from tqdm.auto import tqdm
import os
%pip install tqdm
%pip install anndata

google_drive_dataset_dir = "/content/drive/MyDrive/VCC/datasets/"
dataset_filenames = [
    # "competition_support_set.zip",
    "vcc_data.zip"
]
out_dirs  = [
    # "competition_support_set",
    "vcc_data"
]

for filename, out_dir in zip(dataset_filenames, out_dirs):
  output_path = google_drive_dataset_dir + filename
  os.makedirs(out_dir, exist_ok=True)

  with ZipFile(output_path, 'r') as z:
      for member in tqdm(z.infolist(), desc="Unzipping", unit="file"):
          z.extract(member, out_dir)
!ls vcc_data competition_support_set

Unzipping:   0%|          | 0/4 [00:00<?, ?file/s]

competition_train.h5	       gene_names.csv  k562_gwps.h5  starter.toml
competition_val_template.h5ad  hepg2.h5        k562.h5
ESM2_pert_features.pt	       jurkat.h5       rpe1.h5


In [7]:
!ls vcc_data competition_support_set

competition_support_set:
competition_train.h5	       gene_names.csv  k562_gwps.h5  starter.toml
competition_val_template.h5ad  hepg2.h5        k562.h5
ESM2_pert_features.pt	       jurkat.h5       rpe1.h5

vcc_data:
adata_Training.h5ad  gene_names.csv  pert_counts_Validation.csv


In [8]:
import h5py
import numpy as np
filename = "competition_support_set/k562.h5"

with h5py.File(filename, "r") as f:
  for key in f.keys():
    print(key)
    print(np.array(f[key]))

X
[[0.        1.7592828 0.        ... 3.7345767 2.7359772 5.5159492]
 [0.        0.        0.        ... 3.57988   2.033628  5.394437 ]
 [0.        1.3249131 0.        ... 3.3046005 2.7844076 4.9049053]
 ...
 [0.        0.        0.        ... 3.6838982 3.1109695 4.9110603]
 [0.        0.8855004 0.        ... 3.6001828 2.2560368 5.0871906]
 [0.        1.5953873 0.        ... 3.0742545 2.1816006 5.0639205]]
layers
[]
obs
['UMI_count' 'batch_var' 'cell_barcode' 'cell_type'
 'core_adjusted_UMI_count' 'core_scale_factor' 'gem_group' 'gene'
 'gene_id' 'gene_transcript' 'mitopercent' 'sgID_AB' 'target_gene'
 'transcript' 'z_gemgroup_UMI']
obsm
[]
obsp
[]
uns
[]
var
['_index']
varm
[]
varp
[]


In [ ]:
import anndata
adata = anndata.read_h5ad("vcc_data/adata_Training.h5ad")
adata